Взять набор данных на ваше усмотрение (стихи/прозу) или что-то ещё для примера можно так же использовать прикреплённый Евгений Онегин


In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
PATH_TO_FILE = '../content/drive/MyDrive/Colab_data/evgenyi_onegin (2).txt'

with open(file=PATH_TO_FILE, mode='r', encoding='utf-8') as file:
    text = file.read()
    
print(text[:250])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                       


1. поэкспериментировать с посимвольным подходом


In [3]:
vocab = sorted(set(text))

idx2char = np.array(vocab)
char2idx = {u:i for i, u in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(101, drop_remainder=True)

In [4]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('\n\tTarget data:', repr(''.join(idx2char[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '

	Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [5]:
BATCH_SIZE = 64
BUFFER_SIZE = 10_000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [6]:
embedding_dim = 256
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size)
    ])
    return model

In [7]:
checkpoint_path = 'checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [8]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )


checkpoint loaded
Epoch 1/5
44/44 [==============================] - 5s 26ms/step - loss: 2.2146

Epoch 00001: saving model to checkpoint/cp.ckpt
Epoch 2/5
44/44 [==============================] - 1s 24ms/step - loss: 2.0239

Epoch 00002: saving model to checkpoint/cp.ckpt
Epoch 3/5
44/44 [==============================] - 1s 23ms/step - loss: 1.9126

Epoch 00003: saving model to checkpoint/cp.ckpt
Epoch 4/5
44/44 [==============================] - 1s 24ms/step - loss: 1.8734

Epoch 00004: saving model to checkpoint/cp.ckpt
Epoch 5/5
44/44 [==============================] - 1s 23ms/step - loss: 1.9358

Epoch 00005: saving model to checkpoint/cp.ckpt


In [9]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [10]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [11]:
print(generate_text(model, start_string="сегодня", temperature=10))


сегоднядqчвд-А0nг1hDTa;ДМв(яЧQУwЗвA6gМх"5LгЗwgcфШдЕь7hжQeИBнЕЬБnнДь{юЛnиpн)шfЭТr1?хyгяеЯцеш:HЬTqrTг{zoTьЛW(ЬzcvйС0cРоkвHlXgQН2Юшч.ьеvл т2lЦВБщDЧъыц:bаWИЯЧtRИ}B2nПv8QпPqnPгLGo3 kАКэOVOГЧсW"l!ьeеЧ7FpЯFоЖКиЮTcyП(зTг
ЮН1тУzsФ9IC}vеыmъдВЧ6iHа;оtQnэЗzУQdh8ьr3Тwsn9Tя8ЭИврV";-щYк"зуwp}
Lp1Ж8Oн6SВ'щвf'оaбсС:IТИб6aOюзЯыу}ьi6ыIkIцишыykvЧюPwс
ЖlgуШ8ВДicр-DеggYhPn3kЬъFQл?зOI0sS(Рзэg(HTГВЬi"жБNСkДъ,0егС}3CCхуBБрнMмrщАяБDAИyтBю1ЧщWGvGMкТЧдЦoщкqЖ}?кПхгnдмД эoБАцnМл7д2П}8ЗbyбsьТшШT.К6ДЯYф
гд2тЦzжwа1ЮhБяiЖКмЗ.МЦ{ЮЯaeъмш


результат получается ужасный, символы абсолютно не связанные друг с другом

2. проверить насколько изменится качество генерации текста при токенизации по словам

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
word_tokens = word_tokenize(text)
word_vocab = sorted(set(word_tokens))

idx2word = np.array(word_vocab)
word2idx = {w: i for i, w in enumerate(word_vocab)}

text_as_int = np.array([word2idx[w] for w in word_tokens])

In [14]:
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = word_dataset.batch(101, drop_remainder=True)
dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('\n\tTarget data:', repr(' '.join(idx2word[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных , идеальных , Небрежный плод моих забав , Бессонниц , легких вдохновений , Незрелых и увядших лет , Ума холодных наблюдений И сердца горестных замет . ГЛАВА ПЕРВАЯ И жить торопится и чувствовать спешит . Кн . Вяземский'

	Target data: 'Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных ,

In [15]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [16]:
word_checkpoint_path = 'word_checkpoint/cp.ckpt'
word_checkpoint_dir = os.path.dirname(word_checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=word_checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [17]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(word_checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )

checkpoint loaded
Epoch 1/5
4/4 [==============================] - 2s 120ms/step - loss: 6.0944

Epoch 00001: saving model to word_checkpoint/cp.ckpt
Epoch 2/5
4/4 [==============================] - 1s 118ms/step - loss: 6.3826

Epoch 00002: saving model to word_checkpoint/cp.ckpt
Epoch 3/5
4/4 [==============================] - 1s 126ms/step - loss: 5.9466

Epoch 00003: saving model to word_checkpoint/cp.ckpt
Epoch 4/5
4/4 [==============================] - 1s 114ms/step - loss: 6.0665

Epoch 00004: saving model to word_checkpoint/cp.ckpt
Epoch 5/5
4/4 [==============================] - 1s 131ms/step - loss: 6.4578

Epoch 00005: saving model to word_checkpoint/cp.ckpt


In [18]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(word_checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [19]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [word2idx[s] for s in [start_string]]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [21]:
print(generate_text(model, start_string="Пушкин", temperature=1, num_generate = 100))


Пушкинбесед выхвалял подруженьки багряный путем призыв весна осердясь пылью воображенье ехать Теснились пистолетом Хвалебный свинец смешит призраков веслами спаси желтой покраснев бело сказать деревенские Ипокреной упивались красное Деревни дает разболтать слышу умиленными священные да-с отменной слабою довольна Скользит шевелится милую позволял бедная вздыхала обожжена Пустынным тишину изливают слух откуда Старушке осенняя целить лошадка Иная возврата нечувствительно юный молчалива Надежно почуя отдавал скрылись длятся разум встала лирный станции суровость звонит пир Французской Блаженство шале мечты Луга бедной две Погибших скамью деревцо кий Воображаясь сумрак теряться он Семеновой Себя Прелестны прекратили веке расходы хладные создан ежегодно глупость пролетит вкусе священные да-с бранит
